---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [3]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [73]:
import re
df = [x.rstrip() for x in df]
        #remove all \ns per line
df = pd.Series(df)
df = df.str.lower()

f1 = r'(?P<date>(?P<month>\d{1,2})[/-' '](?P<day>\d{1,2})[/-' '](?P<year>(?:19|20)?\d{2}))'
f2 = r'(?P<date>(?P<day>\d{1,2}) (?P<month>jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z,]* (?P<year>(?:19|20)?\d{2}))'
f3 = r'(?P<date>(?P<month>jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z.]* (?P<day>\d{1,2})[,|\s]* (?P<year>(?:19|20)?\d{2}))'
f4 = r'(?P<date>[\w.]*(?P<month>jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z.,]* (?P<year>(?:19|20)?\d{2}))'
f5 = r'(?P<date>(?P<month>\d{1,2})[/](?P<year>(?:19|20?)\d{2}))'
f6 = r'(?P<date>(?P<year>(?:19|20)\d{2}))'
r1 = df.str.extractall(f1)
r2 = df.str.extractall(f2) 
r3 = df.str.extractall(f3)
r4 = df.str.extractall(f4)
r4 = r4[103:]
r5 = df.str.extractall(f5)
r5 = r5[25:]
r6 = df.str.extractall(f6)
r6 = r6[-45:]

a = r1.merge(r2, how='outer')
b = a.merge(r3, how='outer')
c = b.merge(r4, how='outer')
d = c.merge(r5, how='outer')
e = d.merge(r6, how='outer')

nums = range (0 , 499)
e.drop(e.index[73], inplace = True)
e = e.reset_index()
e.fillna(1, inplace = True)
dic = {'jan':1,'feb':2,'mar':3,'apr':4,'may':5,'jun':6,'jul':7,'aug':8,'sep':9,'oct':10,'nov':11,'dec':12}
e.replace(dic)
del e['index']
e['date']
e['month'] = e['month'].astype(str)
e['day'] = e['day'].astype(str)
e['year'] = e['year'].astype(str)
e['ndate'] = e.month + '-' + e.day + '-' + e.year
e['final'] = pd.to_datetime(e.ndate, errors='coerce')
e['rank'] = e['final'].rank(ascending = True)

#print(r5[-1:])
ans = e.sort_values(['rank']).reset_index()
final = ans['index']
ans

,index,date,month,day,year,ndate,final,rank
0,9,4/10/71,4,10,71,4-10-71,1971-04-10,1.0
1,84,5/18/71,5,18,71,5-18-71,1971-05-18,2.0
2,2,7/8/71,7,8,71,7-8-71,1971-07-08,3.0
3,53,7/11/71,7,11,71,7-11-71,1971-07-11,4.0
4,28,9/12/71,9,12,71,9-12-71,1971-09-12,5.0
5,474,1972,1,1,1972,1-1-1972,1972-01-01,6.0
6,153,13 jan 1972,jan,13,1972,jan-13-1972,1972-01-13,7.0
7,13,1/26/72,1,26,72,1-26-72,1972-01-26,8.0
8,129,06 may 1972,may,06,1972,may-06-1972,1972-05-06,9.0
9,98,5/13/72,5,13,72,5-13-72,1972-05-13,10.0


In [70]:
ans.loc[ans['index'] == 0]

,index,date,month,day,year,ndate,final,rank
272,0,03/25/93,03,25,93,03-25-93,1993-03-25,273.0


In [40]:
def date_sorter():
    
    final
    
    return final